In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../federated")


from notears_libs.utils import set_random_seed, simulate_dag, simulate_parameter, simulate_linear_sem, count_accuracy
from notears_libs.linear import notears_linear, notears_linear_fed

from utils import calculate_metrics

set_random_seed(1)

ModuleNotFoundError: No module named 'notears_libs'

## Notears

#### Dataset generation

In [16]:
n, d, s0, graph_type, sem_type = 5000, 20, 40, 'ER', 'gauss'

adj_mat_true = simulate_parameter(simulate_dag(d, s0, graph_type))

dataset = simulate_linear_sem(adj_mat_true, n, sem_type)
print(f'Shape of dataset: {dataset.shape}')
print(f'Adjacency matrix: {adj_mat_true}')

Shape of dataset: (5000, 20)
Adjacency matrix: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -1.03343447  0.
   0.          0.50541388  0.         -0.67660725  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -1.13215183  0.          0.          0.          0.
   0.          0.         -1.64029786  0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.67867058  1.44043579  0.          0.

#### Without belief aggregation

In [17]:
last_w_est = notears_linear(dataset, lambda1=0.01, loss_type='l2', ground_truth=adj_mat_true)

> 0 SHD = 40
> 1 SHD = 17
> 2 SHD = 5
> 3 SHD = 3
> 4 SHD = 0
> 5 SHD = 2
> 6 SHD = 0
> 7 SHD = 0
> 8 SHD = 0
> 9 SHD = 0
> 10 SHD = 0
> 11 SHD = 0


#### With belief aggregation

In [ ]:
prior = notears_linear_fed(dataset, W_prior=last_w_est, lambda1=0.01, loss_type='l2', ground_truth=adj_mat_true)

> 0 SHD = 40
> 1 SHD = 17
> 2 SHD = 6


In [ ]:
prior = notears_linear_fed(dataset, W_prior=np.ones(800), lambda1=0.1, loss_type='l2', ground_truth=adj_mat_true)

In [ ]:
prior = notears_linear_fed(dataset, W_prior=np.zeros(800), lambda1=0.1, loss_type='l2', ground_truth=adj_mat_true)

#### Evaluation

In [ ]:
calculate_metrics(adj_mat_pred_nobelief, adj_mat_true)

In [ ]:
calculate_metrics(adj_mat_pred_belief, adj_mat_true)

In [ ]:
plt.matshow(adj_mat_pred_nobelief)
plt.matshow(adj_mat_pred_belief)

## IGSP


In [82]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../federated")

from utils import calculate_metrics
from causal_learning import ENCOAlg

import causaldag as cd
np.random.seed(1221)

from causaldag import igsp, unknown_target_igsp, partial_correlation_suffstat, partial_correlation_test
from graphical_models.rand import directed_erdos, rand_weights
from conditional_independence.ci_tests import MemoizedCI_Tester, hsic_test, kci_test, hsic, kci
from conditional_independence import gauss_invariance_suffstat, gauss_invariance_test, MemoizedInvarianceTester
from conditional_independence import hsic_invariance_test, kci_invariance_test

In [115]:
causal_dag_dataset = ENCOAlg.build_global_dataset(obs_data_size=300, int_data_size=1 * 20, num_vars=20,
                                                  graph_type="random", seed=0, num_categs=10, edge_prob=0.6)

INFO - causal_learning.py - 382 - 2021-11-22 13:45:03,833 - Shape of global observational data: (300, 20)
INFO - causal_learning.py - 385 - 2021-11-22 13:45:04,125 - Shape of global interventional data: (20, 1, 20)



In [119]:
def prepare_igsp(causal_dag_dataset, alpha=1e-3, alpha_inv=1e-3, ci_test="hsic"):
    """
        Convert ENCO dataset to IGSP output. 
    """

    obs_samples = causal_dag_dataset.data_obs
    int_samples = causal_dag_dataset.data_int

    iv_samples_list = list()
    targets_list = list()

    for var_idx in range(causal_dag_dataset.adj_matrix.shape[0]):
        for int_sample in int_samples[var_idx]:
            iv_samples_list.append(int_samples[var_idx])
            targets_list.append(var_idx)
    print(f'Shape of obs_samples {obs_samples.shape}')
    print(f'Shape of iv_samples_list ({len(iv_samples_list)}, {iv_samples_list[0].shape})')
    contexts = {i:s for i,s in enumerate(iv_samples_list)}
    invariance_suffstat = {"obs_samples":obs_samples}
    invariance_suffstat.update(contexts)

    ci_tester = MemoizedCI_Tester(hsic_test, obs_samples, alpha=alpha)
    invariance_tester = MemoizedInvarianceTester(hsic_invariance_test, invariance_suffstat, alpha=alpha_inv)
    
    setting_list = [dict(interventions=target) for target in targets_list]
    nodes = set(range(causal_dag_dataset.adj_matrix.shape[0]))
    return ci_tester, invariance_tester, nodes, setting_list 

In [120]:
ci_tester, invariance_tester, nodes, setting_list = prepare_igsp(causal_dag_dataset)

Shape of obs_samples (300, 20)
Shape of iv_samples_list (20, (1, 20))


In [121]:
est_dag, est_targets_list = unknown_target_igsp(setting_list, nodes, ci_tester, invariance_tester)
dag = est_dag.to_amat()[0]

KeyError: 'known_interventions'

In [ ]:
dag